In [1]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Num GPUs Available:  1


In [2]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [3]:
from tensorflow import keras
from tensorflow.keras import layers, models, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import *
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, GlobalMaxPooling2D 
from sklearn.model_selection import train_test_split

In [4]:
pwd

'D:\\projects\\Personal\\Fraudulent-Signature-Detection\\model\\XceptionNetTry1'

In [5]:
# data="../data/cleaned_all_data/"

In [6]:
# df=pd.read_csv("../data/annotated_data.csv")

In [7]:
# images=df["data"].values

In [8]:
# labels=df["annotation"].values

In [9]:
# X=[]
# Y=[]

In [10]:
# for image in images[:]:
#     img=cv2.imread(data+image)
#     img=cv2.bitwise_not(img)
#     #img=img/255
#     X.append(img)

In [11]:
# for label in labels:
#     label_=[label]
#     Y.append(label_)

In [12]:
# X=np.array(X)
# X.shape

In [13]:
# print(X[3])

In [14]:
# Y=np.array(Y)
# Y.shape

In [15]:
from numpy import save, load
X_train=load('../../data/X_train.npy')
y_train=load('../../data/y_train.npy')
X_test=load('../../data/X_test.npy')
y_test=load('../../data/y_test.npy')

In [16]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [17]:
# print(X_train[4])

In [18]:
X_train.shape,y_train.shape

((14644, 256, 512, 3), (14644, 1))

In [19]:
X_test.shape,y_test.shape

((3661, 256, 512, 3), (3661, 1))

In [20]:
# for i in range(14644):
#     X_train[i]=X_train[i]/255

In [21]:
# for i in range(3661):
#     X_test[i]=X_test[i]/255

In [22]:
# save("X_train.npy",X_train)
# save("y_train.npy",y_train)
# save("X_test.npy",X_test)
# save("y_test.npy",y_test)

In [23]:
# print(X_train[4])

In [24]:
input_shape=(256,512,3)

In [25]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 512, 3),
    include_top=False,
)

In [26]:
base_model.trainable = False


In [27]:
inputs = keras.Input(shape=(256, 512, 3))
x=inputs

In [28]:
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
x= keras.layers.Dense(128)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

In [29]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 512, 3)]     0         
                                                                 
 xception (Functional)       (None, 8, 16, 2048)       20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                             

In [30]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 20

In [31]:
base_model=model.fit(X_train[:5000],y_train[:5000], epochs=epochs, validation_data=(X_test[:1000],y_test[:1000]))

Epoch 1/20
157/157 [==============================] - 64s 335ms/step - loss: 0.6170 - binary_accuracy: 0.6534 - val_loss: 0.6474 - val_binary_accuracy: 0.5980
Epoch 2/20
157/157 [==============================] - 52s 331ms/step - loss: 0.5488 - binary_accuracy: 0.7006 - val_loss: 0.5657 - val_binary_accuracy: 0.6780
Epoch 3/20
157/157 [==============================] - 53s 338ms/step - loss: 0.5256 - binary_accuracy: 0.7282 - val_loss: 0.5678 - val_binary_accuracy: 0.6570
Epoch 4/20
157/157 [==============================] - 54s 343ms/step - loss: 0.5058 - binary_accuracy: 0.7364 - val_loss: 0.5520 - val_binary_accuracy: 0.7340
Epoch 5/20
157/157 [==============================] - 54s 344ms/step - loss: 0.5154 - binary_accuracy: 0.7278 - val_loss: 0.5297 - val_binary_accuracy: 0.7250
Epoch 6/20
157/157 [==============================] - 54s 344ms/step - loss: 0.4925 - binary_accuracy: 0.7514 - val_loss: 0.5430 - val_binary_accuracy: 0.7340
Epoch 7/20
157/157 [==========================

In [32]:
model.save("../saved_models/xcep_try1_base1.h5")

D:\projects\Jupyter\gpu-env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
